# _*Reinforcement Learning and Dynamic Optimization*_
## Assignment 3 (2024) - Stock Portfolio Optimization
---
### Students Info
---
##### ID : 2020030055
##### Full Name : Georgios - Marios Tsikritzakis
---
##### ID : 2020030137
##### Full Name : Vasileios Koutsovasilis
---

## Environment

In [ ]:
"""
Implementation for N=3.
"""
import numpy as np

# Define parameters
N = 3         # Number of stocks
c = 0.05      # Transaction fee
gamma = 0.95  # Discount factor gamma

# Expected gains
r_H = [0.1, 0.06, 0.04]  # Gains in state H for each stock
r_L = [0.01, -0.02, 0.03]  # Gains in state L for each stock


# Transition probabilities
P_HH = [0.9, 0.85, 0.8]
P_HL = [0.1, 0.15, 0.2]
P_LH = [0.1, 0.2, 0.25]
P_LL = [0.9, 0.8, 0.75]

"""
Semiography of the states as in the pdf:
(2,1,0,0) == {2 , (1H , 2L , 3L) }
"""

# Define states
states = [(chosen_stock,) + tuple(s) for chosen_stock in range(N) for s in np.ndindex((2,)*N)]
print("States:\n")
print(states)
print("\n")

state_indices = {state: i for i, state in enumerate(states)}
num_states = len(states)
print(f"Number of states: {num_states}\n")

# Define actions
actions = range(N)

# Reward function R(s,a,s')
def reward(state, action, next_state):
    current_stock = state[0]
    next_stock_state = next_state[action + 1]
    if current_stock != action:
        return -c + (r_H[action] if next_stock_state == 1 else r_L[action])
    else:
        return r_H[action] if next_stock_state == 1 else r_L[action]


# Transition probability function
def transition_probability(current_state, action, next_state):
    if action != next_state[0]: return 0

    _, *current_states = current_state
    _, *next_states = next_state
    prob = 1.0
    for i in range(N):
        if current_states[i] == 1:
            prob *= P_HH[i] if next_states[i] == 1 else P_HL[i]
        else:
            prob *= P_LL[i] if next_states[i] == 0 else P_LH[i]
    return prob



States:

[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1), (2, 0, 0, 0), (2, 0, 0, 1), (2, 0, 1, 0), (2, 0, 1, 1), (2, 1, 0, 0), (2, 1, 0, 1), (2, 1, 1, 0), (2, 1, 1, 1)]


Number of states: 24



## Policy Iteration Algorithm

In [ ]:
# Policy Evaluation with V(s)
def policy_evaluation(policy, states, gamma, epsilon=1e-10):
    V = np.zeros(num_states,dtype=np.float64) #V(s) = 0, for every s
    delta = float('inf')  # delta = V - Vold
    while delta > epsilon:
        delta = 0
        # Vold(s) = V(s)
        V_old = V.copy()
        for state in states:
            state_idx = state_indices[state]
            action = policy[state_idx] # policy action
            next_states_for_action = [state for state in states if state[0] == action] # get the next possible states depending on the action
            # Update estimation V(s) = E[ R + gamma * Vold(s') ]
            V[state_idx] = sum(
                transition_probability(state, action, next_state) *
                (reward(state, action, next_state) + gamma * V_old[state_indices[next_state]])
                for next_state in next_states_for_action
            )
            delta = max(delta, abs(V[state_idx] - V_old[state_idx]))
    return V


# Policy Improvement
def policy_improvement(V, states, gamma):
    policy = np.zeros(num_states, dtype=int) # p(s) = 0, for every s
    for state in states:
        state_idx = state_indices[state]
        q_sa = np.zeros(N)
        for action in range(N):
            next_states_for_action = [state for state in states if state[0] == action] # get the next possible states depending on the action
            # Q(s,a) = E[ r + gamma * V_p(s') ]
            q_sa[action] = sum(
                transition_probability(state, action, next_state) *
                (reward(state, action, next_state) + gamma * V[state_indices[next_state]])
                for next_state in next_states_for_action
            )
        policy[state_idx] = np.argmax(q_sa) # p(s) = argmax(Q(s,a))
    return policy


# Policy Iteration
def policy_iteration(states, gamma, epsilon=1e-10):
    policy = np.zeros(num_states, dtype=int) # Policy = [0,0,...] means choose stock 1 always.
    is_policy_stable = False # flag to check if the policy has converged
    iteration = 0

    while not is_policy_stable:
        iteration += 1
        V = policy_evaluation(policy, states, gamma, epsilon)
        new_policy = policy_improvement(V, states, gamma)
        if np.array_equal(policy, new_policy): # if p(s) = p_old(s) the we have converged
            is_policy_stable = True
        policy = new_policy

    print(f"Converged in {iteration} iterations")
    policy_table = {state:policy[state_indices[state]] for state in states}
    return policy_table, V


## Simulation:

In [ ]:
# Run policy iteration
policy, V = policy_iteration(states, gamma)
print("Optimal value function:")
print(V)

print("\nOptimal policy:\n")
for st,act in policy.items():
  state_list = list(st)
  curr_stock = st[0] + 1 # current stock
  new_list = [ str(i+1)+str("H" if s else "L") for i,s in enumerate(state_list[1:])]
  new_list.insert(0,str(curr_stock))
  print(f"State:{new_list}  , action:{act+1}")


Converged in 5 iterations
Optimal value function:
[1.01499155 1.01975132 1.03689254 1.03813567 1.29999143 1.30036099
 1.30285963 1.30305718 1.0093375  1.01975132 1.08689254 1.08813567
 1.24999143 1.25036099 1.25285963 1.25305718 1.0593375  1.06975132
 1.05972756 1.07013297 1.24999143 1.25036099 1.25285963 1.25305718]

Optimal policy:

State:['1', '1L', '2L', '3L']  , action:1
State:['1', '1L', '2L', '3H']  , action:3
State:['1', '1L', '2H', '3L']  , action:2
State:['1', '1L', '2H', '3H']  , action:2
State:['1', '1H', '2L', '3L']  , action:1
State:['1', '1H', '2L', '3H']  , action:1
State:['1', '1H', '2H', '3L']  , action:1
State:['1', '1H', '2H', '3H']  , action:1
State:['2', '1L', '2L', '3L']  , action:3
State:['2', '1L', '2L', '3H']  , action:3
State:['2', '1L', '2H', '3L']  , action:2
State:['2', '1L', '2H', '3H']  , action:2
State:['2', '1H', '2L', '3L']  , action:1
State:['2', '1H', '2L', '3H']  , action:1
State:['2', '1H', '2H', '3L']  , action:1
State:['2', '1H', '2H', '3H']  , 

##Questions to Answer

###Question 1

In [ ]:
import numpy as np

# Define parameters
N = 2         # Number of stocks
c = 0.8      # Transaction fee
gamma = 0  # Discount factor gamma

# Expected gains
r2_H = 0.04
r1_H = 2*r2_H
r_H = [r1_H, r2_H]  # Gains in state H for each stock
r_L = [0.01, -0.02]  # Gains in state L for each stock


# Transition probabilities
P_HH = [0.9, 0.85]
P_HL = [0.1, 0.15]
P_LH = [0.1, 0.2]
P_LL = [0.9, 0.8]

"""
Semiography of the states as in the pdf:
(2,1,0,0) == {2 , (1H , 2L , 3L) }
"""

# Define states
states = [(chosen_stock,) + tuple(s) for chosen_stock in range(N) for s in np.ndindex((2,)*N)]
print("States:\n")
print(states)
print("\n")

state_indices = {state: i for i, state in enumerate(states)}
num_states = len(states)
print(f"Number of states: {num_states}\n")

# Define actions
actions = range(N)

# Reward function R(s,a,s')
def reward(state, action, next_state):
    current_stock = state[0]
    next_stock_state = next_state[action + 1]
    if current_stock != action:
        return -c + (r_H[action] if next_stock_state == 1 else r_L[action])
    else:
        return r_H[action] if next_stock_state == 1 else r_L[action]


# Transition probability function
def transition_probability(current_state, action, next_state):
    if action != next_state[0]: return 0

    _, *current_states = current_state
    _, *next_states = next_state
    prob = 1.0
    for i in range(N):
        if current_states[i] == 1:
            prob *= P_HH[i] if next_states[i] == 1 else P_HL[i]
        else:
            prob *= P_LL[i] if next_states[i] == 0 else P_LH[i]
    return prob

#print(transition_probability([0,1,0,0] , actions[2] , [2,1,0,1] ))
#print(reward([0,1,0,0],0))


States:

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]


Number of states: 8



In [ ]:
# Run policy iteration
policy, V = policy_iteration(states, gamma)
print("Optimal value function:")
print(V)

print("\nOptimal policy:\n")
for st,act in policy.items():
  state_list = list(st)
  curr_stock = st[0] + 1 # current stock
  new_list = [ str(i+1)+str("H" if s else "L") for i,s in enumerate(state_list[1:])]
  new_list.insert(0,str(curr_stock))
  print(f"State:{new_list}  , action:{act+1}")


Converged in 2 iterations
Optimal value function:
[ 0.017  0.017  0.073  0.073 -0.008  0.031 -0.008  0.031]

Optimal policy:

State:['1', '1L', '2L']  , action:1
State:['1', '1L', '2H']  , action:1
State:['1', '1H', '2L']  , action:1
State:['1', '1H', '2H']  , action:1
State:['2', '1L', '2L']  , action:2
State:['2', '1L', '2H']  , action:2
State:['2', '1H', '2L']  , action:2
State:['2', '1H', '2H']  , action:2


###Question 2

In [ ]:
import numpy as np

# Define parameters
N = 2         # Number of stocks
c = 0.05      # Transaction fee
gamma = 0.9  # Discount factor gamma

# Expected gains
r2_H = 0.04
r1_H = 2*r2_H
r_H = [r1_H, r2_H]  # Gains in state H for each stock
r_L = [-0.01, 0.01]  # Gains in state L for each stock


# Transition probabilities
P_HH = [0.9, 0.85]
P_HL = [0.1, 0.15]
P_LH = [0.1, 0.2]
P_LL = [0.9, 0.8]

"""
Semiography of the states as in the pdf:
(2,1,0,0) == {2 , (1H , 2L , 3L) }
"""

# Define states
states = [(chosen_stock,) + tuple(s) for chosen_stock in range(N) for s in np.ndindex((2,)*N)]
print("States:\n")
print(states)
print("\n")

state_indices = {state: i for i, state in enumerate(states)}
num_states = len(states)
print(f"Number of states: {num_states}\n")

# Define actions
actions = range(N)

# Reward function R(s,a,s')
def reward(state, action, next_state):
    current_stock = state[0]
    next_stock_state = next_state[action + 1]
    if current_stock != action:
        return -c + (r_H[action] if next_stock_state == 1 else r_L[action])
    else:
        return r_H[action] if next_stock_state == 1 else r_L[action]


# Transition probability function
def transition_probability(current_state, action, next_state):
    if action != next_state[0]: return 0

    _, *current_states = current_state
    _, *next_states = next_state
    prob = 1.0
    for i in range(N):
        if current_states[i] == 1:
            prob *= P_HH[i] if next_states[i] == 1 else P_HL[i]
        else:
            prob *= P_LL[i] if next_states[i] == 0 else P_LH[i]
    return prob

#print(transition_probability([0,1,0,0] , actions[2] , [2,1,0,1] ))
#print(reward([0,1,0,0],0))

States:

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]


Number of states: 8



In [ ]:
# Run policy iteration
policy, V = policy_iteration(states, gamma)
print("Optimal value function:")
print(V)

print("\nOptimal policy:\n")
for st,act in policy.items():
  state_list = list(st)
  curr_stock = st[0] + 1 # current stock
  new_list = [ str(i+1)+str("H" if s else "L") for i,s in enumerate(state_list[1:])]
  new_list.insert(0,str(curr_stock))
  print(f"State:{new_list}  , action:{act+1}")


Converged in 3 iterations
Optimal value function:
[0.29350022 0.33532126 0.52107809 0.526245   0.34350022 0.38532126
 0.47107809 0.476245  ]

Optimal policy:

State:['1', '1L', '2L']  , action:2
State:['1', '1L', '2H']  , action:2
State:['1', '1H', '2L']  , action:1
State:['1', '1H', '2H']  , action:1
State:['2', '1L', '2L']  , action:2
State:['2', '1L', '2H']  , action:2
State:['2', '1H', '2L']  , action:1
State:['2', '1H', '2H']  , action:1


###Question 3

In [ ]:
import numpy as np
import random
import itertools

# Number of stocks
N = 8
# Discount factor
gamma = 0.9
# Transaction fee
c = 0.05


def generate_rewards(N):
    r_H = np.random.uniform(-0.02, 0.1, N)
    r_L = np.array([np.random.uniform(-0.02, r_H_i) for r_H_i in r_H])
    return r_H, r_L

# Generate rewards ensuring r_L < r_H for each stock
r_H, r_L = generate_rewards(N)

# Randomly assign transition probabilities
P_HL = np.array([0.1 if i < N // 2 else 0.5 for i in range(N)])
P_HH = 1 - P_HL
P_LH = np.array([0.5 if i < N // 2 else 0.1 for i in range(N)])
P_LL = 1 - P_LH

states = [(chosen_stock,) + tuple(s) for chosen_stock in range(N) for s in np.ndindex((2,)*N)]
print("States:\n")
print(states)
print("\n")
state_indices = {state: i for i, state in enumerate(states)}
num_states = len(states)
print(f"Number of states: {num_states}\n")

# Define actions
actions = range(N)


print(f"N = {N}")
print(f"r_H:{r_H}")
print(f"r_L:{r_L}")
print(f"P_HH:{P_HH}")
print(f"P_HL:{P_HL}")
print(f"P_LL:{P_LL}")
print(f"P_LH:{P_LH}")

# Run policy iteration
policy, V = policy_iteration(states, gamma)
print("Optimal value function:")
print(V)

print("\nOptimal policy:\n")
for st,act in policy.items():
  state_list = list(st)
  curr_stock = st[0] + 1 # current stock
  new_list = [ str(i+1)+str("H" if s else "L") for i,s in enumerate(state_list[1:])]
  new_list.insert(0,str(curr_stock))
  print(f"State:{new_list}  , action:{act+1}")

States:

[(0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 1, 1), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 1), (0, 0, 0, 0, 0, 0, 1, 1, 0), (0, 0, 0, 0, 0, 0, 1, 1, 1), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 0, 0, 1, 0, 1, 0), (0, 0, 0, 0, 0, 1, 0, 1, 1), (0, 0, 0, 0, 0, 1, 1, 0, 0), (0, 0, 0, 0, 0, 1, 1, 0, 1), (0, 0, 0, 0, 0, 1, 1, 1, 0), (0, 0, 0, 0, 0, 1, 1, 1, 1), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 1), (0, 0, 0, 0, 1, 0, 0, 1, 0), (0, 0, 0, 0, 1, 0, 0, 1, 1), (0, 0, 0, 0, 1, 0, 1, 0, 0), (0, 0, 0, 0, 1, 0, 1, 0, 1), (0, 0, 0, 0, 1, 0, 1, 1, 0), (0, 0, 0, 0, 1, 0, 1, 1, 1), (0, 0, 0, 0, 1, 1, 0, 0, 0), (0, 0, 0, 0, 1, 1, 0, 0, 1), (0, 0, 0, 0, 1, 1, 0, 1, 0), (0, 0, 0, 0, 1, 1, 0, 1, 1), (0, 0, 0, 0, 1, 1, 1, 0, 0), (0, 0, 0, 0, 1, 1, 1, 0, 1), (0, 0, 0, 0, 1, 1, 1, 1, 0), (0, 0, 0, 0, 1, 1, 1, 1, 1), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 1), (0, 